In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.models import *
from tensorflow.keras.applications import *
from tensorflow.keras.optimizers import *
import tensorflow as tf
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
import os
import numpy as np
np.warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow_addons as tfa
import pandas as pd
from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt

from skimage.filters import sobel
from skimage import color
from sklearn.model_selection import train_test_split
from PIL import Image
from IPython.utils import io
import warnings
warnings.filterwarnings('ignore') # we ignore a RuntimeWarning produced from dividing by zero
import scipy.io as sio
import scipy.ndimage as ndimage
import lmdb, cv2, re, sys
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
#tf.tpu.experimental.initialize_tpu_system(resolver)
#print("All devices: ", tf.config.list_logical_devices('TPU'))
#strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
#%cd '/content'
#gdd.download_file_from_google_drive(file_id='1lLtRBp1wKuAGIQuc8xcXNqQkaMT4SDxg', dest_path='./5.zip', unzip=True)


In [ ]:
import tensorflow as tf
import os
import glob
import PIL
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from sklearn.model_selection import train_test_split

In [ ]:
train_path = '/content/drive/MyDrive/MRI_ACDCA/Dataset2/training'
regions = {'enpi': 3.0 , 'endo': 2.0 , 'rv' : 1.0 }
test_size = 0.2
target_size = (192,192)
crop_size = (128,128)

In [ ]:
def center_crop(img, crop_size = (100,100)):
  w_in, h_in = img.shape
  w_out, h_out = crop_size
  img_clone = np.zeros((w_out, h_out))
  sub_w = int((w_in - w_out)/2)
  sub_h = int((h_in - h_out)/2)
  img_clone = img[sub_w:sub_w+w_out, sub_h:sub_h+h_out]
  return img_clone

In [ ]:
train_file = '/content/drive/MyDrive/MRI_ACDCA/Dataset2/training/patient004'
i = 0 #thay doi cai nay nua
lis_img = glob.glob(train_file+'/*')
list_image = [aa for aa in lis_img if aa.find('frame') != -1 and aa.find('gt') == -1]

In [ ]:
lis_img

In [ ]:
list_image

In [ ]:
z=1 #thay doi cai nay de lay anh 
im = list_image[0]
num = im.find('nii')
im_gt = im[:num-1] + '_gt.' +im[num:]
image = nib.load(im)
label = nib.load(im_gt)
image = image.get_data().astype(np.uint8)
label = label.get_data()
sub_image = image[...,z]
sub_label = label[...,z]
sub_label = np.where(sub_label >= regions['enpi'], 1.0, 0.0).astype(np.float32)

In [ ]:
np.unique(image)

In [ ]:
crop_image = center_crop(image[...,z], crop_size = (128,128))
crop_label = center_crop(sub_label, crop_size = (128,128))

In [ ]:
plt.figure(1)

plt.subplot(121),plt.imshow(sub_image,cmap='gray')
plt.subplot(122),plt.imshow(sub_label,cmap='gray')

In [ ]:
plt.figure(1)

plt.subplot(121),plt.imshow(crop_image,cmap='gray')
plt.subplot(122),plt.imshow(crop_label,cmap='gray')

In [ ]:
x=crop_label 
#x o day gia su la prediction nhe 
w_in, h_in = sub_image.shape
w_out, h_out = (128,128)
out = np.zeros((w_in, h_in))
sub_w = int((w_in - w_out)/2)
sub_h = int((h_in - h_out)/2)
out[sub_w:sub_w+w_out, sub_h:sub_h+h_out]= x

In [ ]:
plt.imshow(out,cmap='gray')

In [ ]:
def get_data(list_file,is_crop = None ,target_size=(192,192), region = regions['endo']):
  
  x_train_list = []
  y_train_list = []
  

  for f in list_file:
    list_fimg = glob.glob(f+'/*')
    list_image = [aa for aa in list_fimg if aa.find('frame') != -1 and aa.find('gt') == -1]

    for im in list_image:
      num = im.find('nii')
      im_gt = im[:num-1] + '_gt.' +im[num:]
      image = nib.load(im)
      label = nib.load(im_gt)
      image = image.get_data().astype(np.uint8)
      label = label.get_data()

      for z in range(image.shape[-1]):
        sub_image = image[...,z]
        sub_label = label[...,z]
        sub_label = np.where(sub_label >= region, 1.0, 0.0).astype(np.float32)
        

        if is_crop:
          sub_label = center_crop(sub_label, crop_size=target_size)
          sub_image = center_crop(image[...,z], crop_size = target_size)

        else:
          sub_image  = PIL.Image.fromarray(sub_image)
          sub_label  = PIL.Image.fromarray(sub_label)
          sub_image = np.array(sub_image.resize(size=(target_size[0],target_size[1],z),resample = PIL.Image.LANCZOS))
          sub_label = np.array(sub_label.resize(size=(target_size[0],target_size[1]),resample = PIL.Image.LANCZOS))

        y_train_list.append(sub_label)
        x_train_list.append(sub_image)
        
  x_ = np.asarray(x_train_list)
  y_ = np.asarray(y_train_list)
  return np.expand_dims(x_, axis = -1), np.expand_dims(y_, axis = -1)





In [ ]:
def get_list_file(path=train_path,rate = 0.3,seed = 42):
  list_file = glob.glob(path+'/*')
  
  total_file = len(list_file)
  test_length = int(total_file*rate)
  train_length = total_file - test_length
  valid_length = int(test_length * 0.5)
  test_length = test_length - valid_length
  np.random.seed(42)
  idx = np.random.permutation(np.arange(total_file))
  train_file = np.asarray(list_file)[idx[:train_length]].tolist()
  test_file = np.asarray(list_file)[idx[train_length:train_length+test_length]].tolist()
  valid_file = np.asarray(list_file)[idx[train_length+test_length:]].tolist()
  return train_file,test_file,valid_file

In [ ]:
train_file,test_file,valid_file = get_list_file(path=train_path)

In [ ]:
train_file = glob.glob('/content/drive/My Drive/acdca_segment/training/*')
test_file = glob.glob('/content/testing/*')

In [ ]:
def save_data(target_size=(192,192),type_file = 'train', list_file = train_file):
  %cd '/content/drive/My Drive/acdca_segment/data_resize'
  for is_crop in [True,False]:
    for j in ['endo','enpi']:
      images , labels = get_data(list_file,is_crop = is_crop ,target_size=target_size, region = regions[j])
      
      images_3D = np.concatenate([images,images,images],axis=-1)
      
      post1 = '_crop_' +str(target_size[1])+'.npy' if is_crop == True else '_' +str(target_size[1])+ '.npy'
      post2 = '_crop_3D_'+str(target_size[1])+ '.npy' if is_crop == True else '_3D_' +str(target_size[1])+ '.npy'
      file_name = j + post1
      file_name2 = j +post2
      np.save(type_file+'_image_'+file_name,images)
      np.save(type_file+'_label_'+file_name,labels)
      np.save(type_file+'_image_'+file_name2,images_3D)


    



In [ ]:
#save_data(type_file='train',list_file=train_file)
#save_data(type_file='test',list_file=test_file)
#save_data(type_file='valid',list_file=valid_file)
save_data(target_size=(128,128),type_file='train',list_file=train_file)
#save_data(target_size=(128,128),type_file='test',list_file=test_file)
#save_data(target_size=(128,128),type_file='valid',list_file=valid_file)

In [ ]:
ls

train_image_endo_128.npy          train_image_enpi_crop_128.npy
train_image_endo_3D_128.npy       train_image_enpi_crop_3D_128.npy
train_image_endo_crop_128.npy     train_label_endo_128.npy
train_image_endo_crop_3D_128.npy  train_label_endo_crop_128.npy
train_image_enpi_128.npy          train_label_enpi_128.npy
train_image_enpi_3D_128.npy       train_label_enpi_crop_128.npy


In [ ]:
def show_sample():
  for i in range(1,20,2):
    plt.figure(i)
    plt.subplot(121), plt.imshow(np.squeeze(train_image_endo[i]),cmap='gray')
    plt.subplot(122), plt.imshow(np.squeeze(train_label_endo[i]),cmap='gray')
    plt.figure(i+1)

    plt.subplot(121), plt.imshow(np.squeeze(train_image_endo_crop[i]),cmap='gray')
    plt.subplot(122), plt.imshow(np.squeeze(train_label_endo_crop[i]),cmap='gray')
  